In [ ]:
import torch
from frontend import MFCCTransform
from utils import get_device
from speechbrain.lobes.models.ECAPA_TDNN import ECAPA_TDNN
from models import WavLM_Base_ECAPA_TDNN

MFCCS = 80
EMBEDDING_SIZE = 192
MODEL = "WavLM-Base-joint_ECAPA-TDNN_Random-Triplet-Mining_Genuine"

device = get_device()
#model = ECAPA_TDNN(input_size=MFCCS, lin_neurons=EMBEDDING_SIZE, device=device)
frozen = False
model = WavLM_Base_ECAPA_TDNN(frozen=frozen, device=device)
model.to(device)

state = torch.load(
    f'../models/{MODEL}_best_model_state.pth')
model.load_state_dict(state)
model.eval()

In [ ]:
from utils import load_deepfake_dataset
from torch.utils.data import DataLoader
from dataloader import ValidationDataset, collate_valid_fn

SAMPLE_RATE = 16000
DOWNSAMPLING_VALID = -1
BATCH_SIZE = 4

train_labels, dev_labels, test_labels = load_deepfake_dataset()

#frontend = MFCCTransform(
#    number_output_parameters=MFCCS, sample_rate=SAMPLE_RATE)
frontend = lambda x: x

validation_dataset = ValidationDataset(
    dev_labels, frontend=frontend, downsampling=DOWNSAMPLING_VALID)
validation_dataset.data_list = validation_dataset.data_list[validation_dataset.data_list["is_genuine"] == 1]
validation_dataloader = DataLoader(validation_dataset, batch_size=BATCH_SIZE, shuffle=True,
                                    drop_last=True, num_workers=4, pin_memory=True, collate_fn=collate_valid_fn)

In [ ]:
from utils import ModelValidator

validator = ModelValidator(validation_dataloader, device)

In [ ]:
validator.validate_model(model, deepfake_eer=False, mlflow_logging=False)